<a href="https://colab.research.google.com/github/luisedchaf/tesis/blob/main/Real_time_MPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPC Control Code for Fertigation 
*Luis Eduardo Chamorro*



14/04/2022

## Libraries

In [6]:
!pip install qpsolvers
!pip install psycopg2

In [8]:
# Import libraries
import statistics
import math                       # Para funciones matemáticas
import pandas as pd               # Para manejo y análisis de estructura de datos
import numpy as np                # Cálculo numérico y el análisis de datos
import matplotlib.pyplot as plt   # Para gráficas
from cvxopt import matrix, solvers
from qpsolvers import solve_qp
import scipy.sparse as spa
import osqp
from scipy import linalg as la
import psycopg2 
from datetime import datetime, timedelta
#pip install quadprog
#import pdb

# Query Database Functions

In [ ]:
def run_query(query=''):
  DB_HOST = '192.68.185.26' 
  DB_USER = 'inv_luis_eduardo' 
  DB_PASS = '12345678*aA' 
  DB_NAME = 'cafiot' 
  conexion = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASS)
  df = pd.read_sql(query,conexion)  # data frame
  conexion.close()
  return df

## Sensor query

In [ ]:
# Code to query the sensors to the database.
# Host in "DATA".
def consulta(year,month,day,hour1,hour2,minute,sensor):
  consulta = """
  SELECT valor1, fecha, topic, s.nombre, key, observacion, color, valor_max, valor_min, tm.nombre, um.nombre
  FROM mediciones AS m 
  INNER JOIN sensores AS s
  ON s.id = m.id_sensor 
  INNER JOIN tipos_mediciones AS tm 
  ON tm.id = s.id_tipo_medicion
  INNER JOIN unidades_medidas AS um 
  ON tm.id_und_medida = um.id
  WHERE key = '{sensor}' -- Temperatura --TMAX TMIN TMED
  AND fecha BETWEEN '{year}/{month}/{day} {hour2}:{minute}:00' AND '{year}/{month}/{day} {hour1}:{minute}:00'
  """.format(year=year,month=month,day=day,hour1=hour1,minute=minute,hour2=hour2,sensor=sensor)
  DATA = run_query(consulta)
  return DATA

In [ ]:
# consult one hour before
now=datetime.now()
now = now-timedelta(hours=5)
now2 = now-timedelta(hours=1)
year = now.year
month = now.month
day = now.day
hour1 = now.hour
hour2 = now2.hour
minute = now.minute

In [ ]:
#print(hour1)
#print(hour2+0.5)

In [ ]:
# Extract from DATA the temperature values
sensor = 'c9478c82-081b-4cbc-b296-107b971d3a18'
temp = consulta(year=year,month=month,day=day,hour1=hour1,minute=minute,hour2=hour2,sensor=sensor)
temp_max = temp.valor1.max()
temp_min = temp.valor1.min()
temp_med = (temp_max+temp_min)/2
print('Temperatura min:{}\nTemperatura Maxima:{}\nTemperatura Media:{}'.format(temp_min,temp_max,temp_med))

In [ ]:
# Extract from DATA the solar radiation values
sensor_rad = '26547499-9f22-465e-abbb-fa1a4769aa04' #radiacion
Radiation1 = consulta(year=year,month=month,day=day,hour1=hour1,minute=minute,hour2=hour2,sensor=sensor_rad) #consult the radiation on the previous hour
Radiation = statistics.mean(Radiation1.valor1) # average radiation
#print('Radiacion:{}'.format(Radiation))

In [ ]:
# Extract from DATA the wind speed values
sensor_wind = '63005737-45c1-4f2f-8a46-9eadf77d2841'
wind_sp = consulta(year=year,month=month,day=day,hour1=hour1,minute=minute,hour2=hour2,sensor=sensor_wind)
wind_avg = wind_sp.valor1.mean()
#print('Wind:{}'.format(wind_avg))

In [ ]:
# Extract from DATA the ambient humidity values
sensor_hum = '93afd49e-9e22-4388-ba1c-126708d88820'
humidity = consulta(year=year,month=month,day=day,hour1=hour1,minute=minute,hour2=hour2,sensor=sensor_hum)
humidity_avg = humidity.valor1.mean()
#print('Humidity:{}'.format(humidity_avg))


In [ ]:
# Extract from DATA the rain values
sensor_pluv = 'd3e889f0-de68-49f7-ad4e-027440a5068d'
Rain = consulta(year=year,month=month,day=day,hour1=hour1,minute=minute,hour2=hour2,sensor=sensor_pluv)
Rtot =Rain.valor1.mean()
#print(Rtot)

In [ ]:
# Extract from DATA the soil moisture values (1)
sensor_moist = '1afa2f25-6043-463a-ac27-8e321a954f56'
moisture1 = consulta(year=year,month=month,day=day,hour1=hour1,minute=minute,hour2=hour2,sensor=sensor_moist)
x_1 =moisture1.valor1.mean()
print(x_1)

30.433333333333334


In [ ]:
# Extract from DATA the soil moisture values (2)
sensor_moist2 = 'f22d6510-6c4a-4582-af89-0b1f025be38b'
moisture2 = consulta(year=year,month=month,day=day,hour1=hour1,minute=minute,hour2=hour2,sensor=sensor_moist2)
x_2 =moisture2.valor1.mean()
print(x_2)

36.245


In [ ]:
# Julian day
day_of_year = datetime.now().timetuple().tm_yday
#print(day_of_year)

## ETo_Penman

### climatological data hosted on drive

**Climatological data hosted on Cafiot**

In [ ]:
z          = 2200             # Elevation above sea level
J          = day_of_year      # Julian day
lat        = 1+(21/60)        # Latitude
long       = -77.29086111     # longitude
t1         = hour2+0.5        # standard hour 
TMAX       = temp_max         # maximum temperature
TMIN       = temp_min         # minimum temperature
TMED       = temp_med         # mean temperature
RAD_SOL    = Radiation**0.0864 # Solar Radiation
VEL_VIENTO = wind_avg          # wind speed
HR_MED     = humidity_avg      # mean humidity

### Parameters ETo

In [ ]:
Delta = (4098*(0.6108*np.exp(17.27*TMED/(TMED+237.3)))/(TMED+237.3)**2)         
P = 101.3*((293-0.0065*z)/293)**5.26
Cp = 1.01*(10**-3)
E  = 0.622 
lambd_a = 2.45
Upsilon = (Cp*P)/(E*lambd_a)
P_Ahr = 1+0.34*VEL_VIENTO
P_Bhr = Delta/(Delta+Upsilon*P_Ahr)
P_Chr = Upsilon/(Delta+Upsilon*P_Ahr)
P_Dhr = (900/(TMED+273))*VEL_VIENTO
# Parameter E
# Vapor saturation pressure at maximum air temperature.
e_0Tmax=(0.6108*math.exp((17.27*TMAX)/(TMAX+237.3)))
# Vapor saturation pressure at minimum air temperature
e_0Tmin=(0.6108*math.exp((17.27*TMIN)/(TMIN+237.3)))
# Mean saturation vapor pressure
eshr = (e_0Tmax+e_0Tmin)/2
# Vapor saturation pressure at mean air temperature
e0_Tmedhr = 0.6108*np.exp((17.27*TMED)/(TMED+237.3)) 
# Actual vapor pressure
eahr = (HR_MED/100)*e0_Tmedhr
# Parameter E
P_Ehr = (eshr-eahr)

# Parameter F
# Relative distance to the sun inverse Earth
dr = 1+0.033*math.cos(((2*math.pi)/365)*J)
# Solar Declination
ds = 0.409*math.sin((2*math.pi/365)*J-1.39)

# Calculation of extraterrestrial radiation per hour (Ra)

# Solar constant
Gsc = 0.082
# Latitude in Radians
lat_rad = (math.pi/180)*lat

# Seasonal Correction
b = 2*math.pi*((J-81)/364)
Sc = 0.1645*math.sin(2*b)-0.1255*math.cos(b)-0.025*math.sin(b)
# Solar angle
w = (math.pi/12)*((t1+0.06667)*((75-77)+Sc)-12)
# Solar radiation angles at the beginning and at the end of the day
w1 =(w-(math.pi/24))
w2 =(w+(math.pi/24))
# Ra
Ra = (((12*60/math.pi)*Gsc*dr*((w2-w1)*math.sin(lat_rad)*math.sin(ds)+math.cos(lat_rad)*math.cos(ds)*(math.sin(w2)-math.sin(w1)))))
# Solar radiation on a clear day (Rso)
Rso = (0.75 + 2*10**-5*z)*Ra
# Net solar or shortwave radiation (Rns)
Rns = 0.77*RAD_SOL
# Net longwave radiation (Rnl)
Rs = RAD_SOL                   
Rnl = 4.903*10**-9*(((TMAX)**4 + (TMIN)**4)/2)*(0.34-0.14*math.sqrt(eahr))*(1.35*(Rs/Rso)-0.35) 
P_Fhr = Rns-Rnl

#Parameter G  
P_Ghr = 0.5*P_Fhr

#Penman Monteith Application
ETo = ((0.408*Delta)*(P_Fhr-P_Ghr)+lambd_a*P_Dhr*P_Ehr)/(Delta+lambd_a*P_Ahr)

In [ ]:
print(ETo)

3.3570292336101706


# CONTROL MPC

### Parameters

In [112]:
# Parameters
c1 = 0.05 
c2 = 0.4  
c3 = 1.2  
c4 = 1   
tm = 60/(60*24)   
Da = 0.57
h  = 30 

### state space with 2 states

In [113]:
A=np.matrix([[1-tm*(c1+c2),c2*tm],[tm*c2,1-tm*(c1+c2)]])  # state matrix
B=((tm*24*10)/(Da*h))*np.diag([1,1])                      # input matrix
E=((tm*10)/(Da*h))*np.matrix([[-c3,c4],[-c3,c4]])         # disturbance matrix
C=np.identity(2)                                          # output matrix
x_1 = 34                                                  # only to simulate
x_2 = 28                                                  # only to simulate

### Initial conditions

In [116]:
x=np.matrix(np.array([[x_1],[x_2]]))  
u=np.matrix(np.array([[0],[0]]))
#Eto = ETo
#Rtot = Rtot
Eto = 3.35                             #only to simulate    
Rtot = 0                               #only to simulate
dis=np.matrix(np.array([[Eto],[Rtot]])) 
X=x

In [117]:
# Matrix generation for problem QP 
N=2                                     # Prediction horizon
Q=np.array([[6,0],[0,2]])               # Q to weigh the states
R=np.array([[15,0],[0,15]])             # R to weigh the inputs
P = la.solve_discrete_are(A, B, Q, R)   #Riccati

### Sistem Constrains

In [119]:
# Restricciones
vmax=np.array([71,71,9,9,11*tm,10*tm])      # (Hmax1, Hmax2, umax1, umax2, dumax1, dumax2)  
vmin=np.array([23,23,0,0,-11*tm,-10*tm])    # (Hmin1, Hmin2, umin1, umin2, dumin1, dumin2)
ic0 =np.array([2,1,1/2])                    #  order of vmax and vmin inputs

### QP Matrices Function

In [120]:
def QP_matrices_curso(A,B,C,N,Q,R,P,vmin,vmax,ic0):    
  n =np.shape(B)[0]
  m =np.shape(B)[1]
  E=B
  dmax=0
  dmin=0
  e=np.shape(E)[0]   
  M=N
  
  DeltaM = []
  for i in range(len(vmax)):
      DeltaM=np.append(DeltaM,np.kron(np.ones(N),vmax[i]), axis=0)
  Deltam =[]
  for i in range(len(vmin)):
      Deltam=np.append(Deltam,np.kron(np.ones(N),vmin[i]), axis=0)
  
  # max and min of the perturbation d
  dmax = np.kron(np.ones(M),dmax)
  dmin = np.kron(np.ones(M),dmin)
  
  [H,F,Fd,Gamma,Ommega,Phi,Psi]=RHCmatrices(A,B,C,E,P,Q,R,N,N)
  CC = np.kron(np.diag(np.ones(M)),C)
  phic=[]
  L=[]
  Lc=[]
  Lu=[]
  DeltaMc=[]
  Deltamc=[]
 
  for i in range(len(ic0)):
    ic=ic0[i]    
    #######################################################  
    if ic==1:                       #input constraints
      phi = np.identity(M*m)
      L = np.zeros((M*m,n))
      rho = np.zeros((M*m,M*e))       #JZF
    #########################################################
    if ic==2:                      #state constraints (JZF)
        phi = Gamma
        L = Ommega
        if len(Phi)==0:
          DeltaM = DeltaM
          Deltam = Deltam
        else:
          rho=[]
    ###############################################
    
    if ic==0:                       #output constraints
      phi = CC*Gamma[0:M*n,0:M*m]
      L = CC*Ommega[0:M*n,:]
      if len(Phi)==0:
        DeltaM = DeltaM
        Deltam = Deltam
      else:
        rho=[]
    ###################################################
    if ic==1/2:                                       #rate constraints  ATTENTION it has to be executed at the end
        D1 = np.zeros((M,1))
        D1[0] = 1
        D1 = np.kron(D1,np.identity(m))
        D2 = np.zeros((M*m,M*m))
        D2[m:M*m,0:(M*m-m)] = np.kron(np.identity(M-1),np.identity(m))
        phi = np.identity(M*m)-D2
        L = np.zeros((M*m,n))        
        Lu = np.append((np.zeros((len(phic),m))), -D1, axis=0)        
        F = np.append(np.zeros((M*m,m)),F,axis=1)           
      #####################################################
    if len(phic)==0:
      phic=phi
    else:   
      phic = np.append(phic, phi,axis=0)    
    if len(Lc)==0:
      Lc = L
    else:
      Lc = np.append(Lc, L, axis=0)        
  Lc = np.append(Lu, Lc, axis = 1)  
  return (H,F,Fd,phic,Lc,Deltam,DeltaM)

### RHC matrices

In [121]:
def RHCmatrices(A,B,C,E,P,q,r,N,M):      
  nx = np.shape(B)[0]
  nx = np.shape(B)[1]
  e=np.shape(E)[0]
  
  #luisRH = Eqp
  #Definition of the matrices
  Ommega = A
  for i in range(2,M+1):
    Ommega = np.append(Ommega,A**i,axis=0)
  

  gamma = np.zeros((M,M)) + np.diag(np.ones(M-1),k=-1)
  gamma = np.kron(gamma,np.identity(nx))
  Gamma = gamma*Ommega
  
  for i in range (2,M+1):
    Gamma = np.append(Gamma, (np.linalg.matrix_power(gamma,i))*Ommega,axis=1)

  Gamma = (Gamma+np.identity(M*nx))*np.kron(np.identity(M),B) 

  # Phi Matrix

  if len(E)==0:
    Phi = []

  else:
    phi = np.zeros(M) + np.diag(np.ones(M-1),k=-1)
    phi = np.kron(phi,np.identity(nx))
    Phi = phi*Ommega
    for i in range(2,M+1):
      Phi = np.append(Phi, (np.linalg.matrix_power(phi,i))*Ommega,axis=1)#Phi = np.append(Phi, (phi**i)*Ommega,axis=1)  
       
    Phi = (Phi + np.identity(M*nx))*np.kron(np.identity(M),E) 
  
  # Psi Matrix
  psi = np.zeros(M) + np.diag(np.ones(M-1),k=-1)
  psi = np.kron(psi,np.identity(nx))
  Psi = psi*Ommega
  for i in range(2,M+1):
    Psi = np.append(Psi, (np.linalg.matrix_power(psi,i))*Ommega,axis=1)      
  
  Psi = (Psi + np.identity(M*nx))*np.kron(np.identity(M),np.identity(np.size(A,1))) 

  # Q matrix
  Q = np.kron(np.identity(M),q)

  # P Matrix in Q

  # Get Matrix Sizes 
  mq, sq = Q.shape
  mp, sp = P.shape

  # Assigning matrix portion
  Q[mq-mp:mq , sq-sp:sq] = P
  #Q(nx*N-len(P)+1:nx*N,nx*N-len(P)+1:nx*N) = P

  # R Matrix
  R = np.kron(np.identity(M),r)

  # Matrices used in the u_opt explicit solution.
  H = np.transpose(Gamma)*Q*Gamma+R                
  K1 = np.transpose(Gamma)*Q*Ommega
  
  if len(E)!=0:                                      
    K2 = np.transpose(Gamma)*Q*Phi
  else:
    K2=[]    
  return (H,K1,K2,Gamma,Ommega,Phi,Psi)

In [122]:
[H,F,Fd,phic,Lc,Deltam,DeltaM]=QP_matrices_curso(A,B,C,N,Q,R,P,vmin,vmax,ic0)

### References

In [140]:
# References
us=np.array([[0.19],[0.22]])
xs=np.array([[42],[42]])
Us=np.kron(np.ones((N,1)),us)         # Constant reference vector for N instants
um1=us                                # Control value at instant -1
x0=np.array([[x_1],[x_2]])            # Initial conditions
Xe0=np.append(um1,x0,axis=0)
Xes=np.append([[0]],xs,axis=0)        # Vector of states extended by constraint du

### Quadprog

In [142]:
def quadprog(P, q, G=None, h=None, A=None, b=None,
             initvals=None, verbose=True):
    l = -np.inf * np.ones(len(h))
    if A is not None:
        qp_A = spa.vstack([G, A]).tocsc()
        qp_l = np.hstack([l, b])
        qp_u = np.hstack([h, b])
    else:  # no equality constraint
        qp_A = G
        qp_l = l
        qp_u = h
    model = osqp.OSQP()
    model.setup(P=P, q=q,
                A=qp_A, l=qp_l, u=qp_u, verbose=verbose)
    if initvals is not None:
        model.warm_start(x=initvals)
    results = model.solve()
    return results.x, results.info.status

### Simulation

In [143]:
x=x0                                         # Initial conditions
Xresultado=np.array(x0)
Uresultado=np.array(um1)
H1 = spa.csc_matrix(H)
G1 = spa.csc_matrix(np.append(phic,-phic,axis=0))
q0 = np.ones(4)
Xe0=np.append(um1,x,axis=0)
Xes=np.append(us,xs,axis=0)
subs=np.subtract(Xe0,Xes)
f=np.dot(F,subs)-np.dot(H,Us)
Uopt, xx=quadprog(H1,f,G1,np.append(DeltaM-np.ravel(Lc*Xe0), -Deltam+np.ravel(Lc*Xe0),axis=0), verbose=False)
u=np.transpose(np.matrix(Uopt[0:2]))          # simulates the controller
xsig=A*x+B*u +E*dis                           # simulates the real process
x=xsig
um1=u 
Xresultado=np.append(Xresultado, x, axis = 1) # records results in the states
Uresultado=np.append(Uresultado, u, axis = 1) # records the results of inputs